# NeuroFlow Take-Home Project

## Content

[**Part1**](#part1)  

[I. EDA](#eda)  

[II. Assumption](#assumption)

[III. Patterns over time](#patterns)
- [user1](#user1)
- [user2](#user2)  

[IV. Correlation](#correlation)    

[V. Improvement](#improvement) 

[**Part2**](#part2)

[I. Question 1](#question1)

[II. Question 2](#question2)

## <a name = part1></a>Part 1

 ### <a name = eda></a>EDA

In [196]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objects as go
import datetime
import pytz

In [197]:
data = pd.read_csv('subj_measures.csv')

In [91]:
data.head()

,date,user_id,type,value
0,2019-06-08T23:19:34.418Z,2348,mood,2.76
1,2019-06-13T16:33:34.399Z,5232,sleep,2.44
2,2018-12-26T14:24:00.436Z,4209,sleep,1.88
3,2019-07-17T20:11:23.792Z,2802,mood,2.20
4,2019-05-09T17:27:50.900Z,2025,mood,4.00


In [92]:
#unique user
data.user_id.unique()

array([2348, 5232, 4209, 2802, 2025, 1372, 3782, 2729,  939, 6450, 7162,
       6928, 1260, 1342, 2012, 2666, 1044, 1522, 2432, 1714, 1074, 4612,
       1371, 4500, 3182, 3473, 2311, 6445, 1140, 8355, 7618, 1822, 2928,
       1514, 7209, 7129, 2435, 1513, 7328, 1673, 3314, 6876, 6984,  884,
       1471, 7702, 6510, 5141,  650,  603, 2972, 5333, 1411,  597, 7556,
       1309, 1506, 3450, 1109, 7566, 2497, 7019, 5085, 2283, 6299,  970,
       3246, 3044, 7707, 7329, 1269, 2554, 2703, 1487, 3435, 1208, 8086,
       2271, 2762, 3743, 1271, 7468, 1366, 2198, 1214, 1534, 5924, 3609,
       5497, 4330, 1173, 1972,  652, 7176, 7652,  932, 8371, 2592, 8170,
       2020])

In [93]:
#average score for each type 
data.groupby('type')['value'].mean()

type
anticipatoryStress    2.495910
mood                  2.404216
ruminationStress      2.353558
sleep                 2.404878
Name: value, dtype: float64

In [239]:
#average score of each user for each type 
data.groupby(['user_id','type'])['value'].mean()

user_id  type              
597      anticipatoryStress    2.472315
         mood                  2.401146
         ruminationStress      2.122111
         sleep                 2.585581
603      anticipatoryStress    2.438136
                                 ...   
8355     sleep                 2.177179
8371     anticipatoryStress    1.940000
         mood                  2.586716
         ruminationStress      2.000000
         sleep                 1.440000
Name: value, Length: 370, dtype: float64

In [240]:
data.groupby(['user_id','type'])['value'].count()

user_id  type              
597      anticipatoryStress     3
         mood                  23
         ruminationStress       3
         sleep                 20
603      anticipatoryStress     5
                               ..
8355     sleep                 17
8371     anticipatoryStress     2
         mood                  14
         ruminationStress       1
         sleep                  2
Name: value, Length: 370, dtype: int64

###  <a name = assumption></a>Assumption

1. The total number of data is enough to be generalized
2. Each patient has been constantly provided therapy during the date
3. Every patient recieves the same therapy.

###  <a name = patterns></a>Patterns over time

This part is focusing on individual data to track each patient's subjective metric. Therefore, each user will be extracted and identified. As an example, `user_id = 2348` and `user_id = 603` were selected as `user1` and `user2` and visualized.  
The methods and insights in this part can be used to identify other patients.

 #### <a name = eda></a> User1

In [96]:
#user1: user_id = 2348
user1 = data[data.user_id == 2348]
user1.type.unique()

array(['mood', 'sleep', 'ruminationStress'], dtype=object)

In [97]:
user1.loc[:,'date'] = pd.to_datetime(user1.date)

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [99]:
#remvoe local time
user1.date = user1.date.apply(lambda x: x.tz_localize(tz = None))

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [354]:
user1['date'] = pd.to_datetime(user1['date'], format='%Y-%m-%d %H:%M:%S.%f')
# set minute, second and microsecond as zero; trace by hour
user1.loc[:,'date'] = user1['date'].map(lambda x: x.replace(microsecond=0))
user1.loc[:,'date'] = user1['date'].map(lambda x: x.replace(second=0))
user1.loc[:,'date'] = user1['date'].map(lambda x: x.replace(minute=0))

In [103]:
#sort values by date
user1 = user1.sort_values('date')

In [104]:
#Trace for user1
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=user1.date,
                y=user1.loc[user1.type == 'sleep']['value'],
                name="sleep",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=user1.date,
                y=user1.loc[user1.type == 'mood']['value'],
                name = 'mood',
                line_color='dimgray',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=user1.date,
                y=user1.loc[user1.type == 'ruminationStress']['value'],
                name = 'ruminationStress',
                line_color='green',
                opacity=0.8))



fig.update_layout(title_text="User_id: 2348")
fig.show()

This graph is showing overal tarce of each type: sleep, modd, ruminationStress. User_id = 2348 was selected as user1 as an example of individual analysis. Sleep and mood are not proportional but somewhat correlated. Also, the user tend to measure sleep and mood at the same time. This graph can be used for identifying patterns of each patient's subjective metrics over time. In addition, the patient will be able to see if therapy is making a difference.

In [278]:
#create 'hour' column 
user1['hour']= user1.date.dt.hour
user1.head()

,date,user_id,type,value,hour
5660,2019-04-07 00:00:00,2348,sleep,3.040000,0
6049,2018-12-29 06:00:00,2348,ruminationStress,1.720000,6
2153,2019-02-14 06:00:00,2348,mood,3.714381,6
3841,2018-10-11 12:00:00,2348,sleep,3.000000,12
3516,2018-12-04 12:00:00,2348,mood,4.000000,12


In [176]:
#hourly average value for each type
user1_hourly_avg = pd.DataFrame(user1.groupby(['hour','type'])['value'].mean())
user1_hourly_avg

value
hour type                      
0    sleep             3.040000
6    mood              3.714381
     ruminationStress  1.720000
12   mood              4.000000
     sleep             3.000000
13   sleep             2.434583
14   mood              3.956125
     sleep             1.940266
15   mood              2.722993
16   sleep             3.816997
18   mood              2.360000
19   sleep             4.000000
20   sleep             2.108406
23   mood              2.339003

In [167]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=user1.hour.unique(),
                y=user1.loc[user1.type == 'sleep'].groupby('hour')['value'].mean(),
                name="sleep",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=user1.hour.unique(),
                y=user1.loc[user1.type == 'mood'].groupby('hour')['value'].mean(),
                name = 'mood',
                line_color='dimgray',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=user1.hour.unique(),
                y=user1.loc[user1.type == 'ruminationStress'].groupby('hour')['value'].mean(),
                name = 'ruminationStress',
                line_color='green',
                opacity=0.8))

fig.update_layout(title_text="User_id: 2348")
fig.show()

Furthermore, it is significant to identify patient's patterns in hourly condition. In other words, the graph is useful to figure out specific time window that indicates lower condition level. Therefore, the question whether the measurement submission time affects the level of condition can be answered. For example, user1 (user_id = 2348) is showing decrease in value of mode afternoon. The data is grouped by hour and took an average. The result can be vary by patients 

 #### <a name = eda></a>User 2

In [235]:
user2 = data.loc[data.user_id == 603]

In [355]:
user2.loc[:,'date'] = pd.to_datetime(user2.date)
user2.date = user2.date.apply(lambda x: x.tz_localize(tz = None))
user2['date'] = pd.to_datetime(user2['date'], format='%Y-%m-%d %H:%M:%S.%f')
user2.loc[:,'date'] = user2['date'].map(lambda x: x.replace(microsecond=0))
user2.loc[:,'date'] = user2['date'].map(lambda x: x.replace(second=0))
user2.loc[:,'date'] = user2['date'].map(lambda x: x.replace(minute=0))
user2 = user2.sort_values('date')

In [303]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=user2.date,
                y=user2.loc[user2.type == 'sleep']['value'],
                name="sleep",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=user2.date,
                y=user2.loc[user2.type == 'mood']['value'],
                name = 'mood',
                line_color='dimgray',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=user2.date,
                y=user2.loc[user2.type == 'ruminationStress']['value'],
                name = 'ruminationStress',
                line_color='green',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=user2.date,
                y=user2.loc[user2.type == 'anticipatoryStress']['value'],
                name = 'anticipatoryStress',
                line_color='red',
                opacity=0.8))



fig.update_layout(title_text="User_id: 4209")
fig.show()

Based on this graph, user2 is showing that mood is not necessarily correlated to sleep. In addition, most of users including user1 and user2 tend to stop submitting measurement for `Stress` in the early stage. `ruminationStress` and `anticipatoryStree` might not be informative enough in the dataset.  

Since the patients recieve notifications only for sleep and mood, patients are more likely to rate those two elements. 
Sending a notification for Stress level during the day might help this problems

###  <a name = correlation></a>Correlation between `Sleep` and `Mood`

Looking at  user1 and user2 as a sample, **correlation strength** between sleep and mood can be significantly different. If the correlation strength can be identified for each patient, this information can be useful to suggest a solution to imporve patients' mood or sleep.  

In [319]:
#correlation for each user
user_list = data.user_id.unique()
cov_list = []
for i in user_list:
    samp = data.loc[data.user_id == int(i)]
    s1 = pd.Series(list(samp.loc[samp.type == 'sleep']['value']))
    s2 = pd.Series(list(samp.loc[samp.type == 'mood']['value']))
    cov_val = s1.cov(s2)
    cov_list.append(cov_val)
cov_frame=pd.DataFrame(cov_list,user_list)
cov_frame = cov_frame.rename(columns={0:'cov'})
cov_frame.sort_values(by='cov')

,cov
3743,-1.317682
2554,-0.853085
7566,-0.838508
1972,-0.681990
7618,-0.662453
...,...
932,0.638371
970,0.639043
7556,0.705498
7707,0.748195


This table is representing that strength of correlation between sleep and mood is vary by each patient. For example, `user_id = 1514` is showing stong **positive** correlation while `user_id=3747` is shoing strong **negative** correlation. Based on this information, different therapy can be applied.

#### Average of Total Data

In this part, the question is when patients are more likely to participate in the measurement and if it affects on patients' condition

In [279]:
total_avg = data.copy()
total_avg.date = pd.to_datetime(total_avg.date)
total_avg.date = total_avg.date.apply(lambda x: x.tz_localize(tz = None))
total_avg['date'] = pd.to_datetime(total_avg['date'], format='%Y-%m-%d %H:%M:%S.%f')
total_avg.loc[:,'date'] = total_avg['date'].map(lambda x: x.replace(microsecond=0))
total_avg.loc[:,'date'] = total_avg['date'].map(lambda x: x.replace(second=0))
total_avg.loc[:,'date'] = total_avg['date'].map(lambda x: x.replace(minute=0))
total_avg.head()

,date,user_id,type,value
0,2019-06-08 23:00:00,2348,mood,2.76
1,2019-06-13 16:00:00,5232,sleep,2.44
2,2018-12-26 14:00:00,4209,sleep,1.88
3,2019-07-17 20:00:00,2802,mood,2.20
4,2019-05-09 17:00:00,2025,mood,4.00


In [328]:
#create month column
total_avg['month'] = total_avg.date.dt.month

#count for each month
mon_ct = pd.DataFrame(total_avg.groupby('month')['date'].count())

In [335]:
total_avg = total_avg.sort_values(by='month')

In [327]:
#graph for montly total count 
fig = px.bar(x=mon_ct.index, y=mon_ct.date)
fig.update_layout(title_text='Montly Mesaurement Count')
fig.show()

This bar graph is represeting that patients respond the least on Septempber. Therefore, data from September can be skewed. 

In [352]:
total_avg = total_avg.sort_values('date')
total_avg['hour'] = total_avg.date.dt.hour
#total_avg = pd.DataFrame(total_avg.groupby(['hour','type'])['value'].mean())
total_avg = total_avg.sort_values('hour')
total_avg = total_avg.sort_values('date')
total_avg = total_avg.sort_values('month')

In [353]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=total_avg.month.unique(),
                y=total_avg.loc[total_avg.type == 'sleep'].groupby('month')['value'].mean(),
                name="sleep",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=total_avg.month.unique(),
                y=total_avg.loc[total_avg.type == 'mood'].groupby('month')['value'].mean(),
                name = 'mood',
                line_color='dimgray',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=total_avg.month.unique(),
                y=total_avg.loc[total_avg.type == 'anticipatoryStress'].groupby('month')['value'].mean(),
                name = 'anticipatoryStress',
                line_color='green',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=total_avg.month.unique(),
                y=total_avg.loc[total_avg.type == 'ruminationStress'].groupby('month')['value'].mean(),
                name = 'ruminationStress',
                line_color='red',
                opacity=0.8))

fig.update_layout(title_text="Average Hourly pattern")
fig.show()

This result can be related to the previous graph. From this graph, data from August has the strongest negative correlation between `sleep` and `mood`, or `ruminationStress` and `mood`. Significant low level of mood and high stress might lead to decrease in participation of the measurement. In order to figure out figure out the distinctive factor on August, more information will be needed. 

###  <a name = improvement></a>Improvement

In order to improve the analysis, information of `therapy_date` will be useful to see how much the therapy affects to patients conditions. With `therapy_date`, various analysis can be conducted
In addition `weather` information is frequently used in healthcare data. Weather can significantly effect on patients' mood, sleep or stress level.

## Part 2

####  <a name = question1></a>Question 1

In [356]:
q1='''
SELECT 
    EXTRACT(YEAR_MONTH from e.exercise_completion_date) finish_date,
    EXTRACT(YEAR_MONTH from u.created_at) start_date,
    COUNT(*) 
FROM exercises e
INNER JOIN(
            SELECT * 
            FROM user u
            GROUP BY start_date) u ON u.user_id = e.user_id
WHERE finish_date == start_date

'''

#### <a name = question2></a> Question 2

In [ ]:
q2 = '''
SELECT *
FROM provider
INNER JOIN (
            SELECT * FROM phq9
            WHERE score > 20
           ) phq9 ON phq9.provider_id = provider.provider_id
ORDER BY organization_name DESC
LIMIT 5
''''''